# Export top of the book & trades using dedicated service - F#

### Overview
Retrieving trades and top of the book data points over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request simultaneously trades and top of the book and illustrate market activity over a look back period.

<div class="alert alert-block alert-info">
    <b>Note:</b> To highlight individual tradres among market activity. Please refer to <i><a href="">bestex-pointintime</a></i> and <i><a href="">bestex-interval</a></i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** trades over the time.

### Inputs/outputs
Trades and best Bid/Ask extraction sample requires instrument's identifier, date time intervals as per inputs and returns top fo the book the occured trades data points.

### Services used
This sample uses *gRPC requests* in order to retrieve trades and top of the book from the hosted service. The queried endpoint in this script are:
* *TickTradesAndBookService*: to directly retrieve trades and top of the book data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    
***

# Run Top of the book & Trades extraction sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget:CsvHelper"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Tick.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
// csv extraction
open CsvHelper 
open CsvHelper.Configuration
open System.IO
open System.Globalization
open FSharp.Control.Tasks.V2

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
let token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN")
let api = System.Environment.GetEnvironmentVariable("GRPC_APIS")

// Add token to the request header
let headers = new Metadata()
headers.Add("Authorization", $"Bearer {token}")

// Define API entry
let channel = GrpcChannel.ForAddress($"https://{api}")

In [ ]:
// Display authentication token 
token
|> display

### Step 3: Create and process request
To request *tick trades and book* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [ ]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Time period delimitation

In [ ]:
// Create time intervals
let start = new DateTime(2021, 03, 05)
let end' = new DateTime(2021, 03, 05)
let dateIntervals = 
    new DateInterval(
        StartDate = new Date ( Year = start.Year, Month = start.Month, Day = start.Day ),
        EndDate = new Date ( Year = end'.Year, Month = end'.Month, Day = end'.Day )
    )

// Build the market data request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
let timeInterval = 
    new TimeInterval(
        StartTime = new TimeOfDay ( Hours = 14, Minutes = 00, Seconds = 00 ),
        EndTime = new TimeOfDay ( Hours = 15, Minutes = 15, Seconds = 00)
    )

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

The following code snippet create the market data request and instantiate the service:

In [ ]:
// Generate the tick data request
let request = new TickTradesAndBookRequest ( Constraints = constraints )
let identifier = new Identifier ( Exchange = exchange, Ticker = ticker )
request.Identifiers.Add( [| identifier |]) 

// Instantiate the tick data service
let service = new TickTradesAndBookService.TickTradesAndBookServiceClient(channel)

### Step 4: Export data

#### 4.1 Retrieve top of the book & trades

The following code snippet stores data points for *trades*, *best bid* and *best ask*:

In [ ]:
// Process tick data stream service: get price / size / timestamp
open FSharp.Control.Tasks.V2
let data = System.Collections.Generic.List<TickTradesAndBookResponse>()
// from https://docs.microsoft.com/en-us/archive/msdn-magazine/2019/november/csharp-iterating-with-async-enumerables-in-csharp-8
// there is probably a more idiomatic way
let t = 
    task {
        let call = service.TickTradesAndBook(request, headers)
        let enumerator = call.ResponseStream.ReadAllAsync().GetAsyncEnumerator()
        let mutable moveNext = false
        try
            let! moveNext'  = enumerator.MoveNextAsync()
            moveNext <- moveNext'
            while moveNext do
                data.Add enumerator.Current
                let! moveNext'  = enumerator.MoveNextAsync()
                moveNext <- moveNext'
        finally 
            if ((isNull enumerator)|>  not) then
                //do! enumerator.DisposeAsync().AsTask()
                let disp = task { do! enumerator.DisposeAsync() }
                disp.Wait()
            else
                ()
    }

t.Wait()

//data
//|> display

The following code snippet displays request results count:

In [ ]:
// Display trade and top the book data points
$"Total extracted data: {data.Count:N0} data points"
|> display

#### 4.2 Export top of the book & trades

The following code snippets export results to *csv file*:

In [ ]:
type TradeAndBookData = { TimeStamp : DateTime; TradePrice : Nullable<float>; TradeSize : Nullable<int64>;  AskPrice : Nullable<float>; AskSize : Nullable<int64>; BidPrice : Nullable<float>; BidSize : Nullable<int64> }

let choose (d : TickTradesAndBookResponse) =
    if (d.Data |> isNull) then
        None
    else 
        Some d.Data

let convert (d: TradeAndBook) = 
    if isNull d.Trade then
        { 
            TimeStamp = d.TimeStamp.ToDateTime()
            TradePrice = Nullable()
            TradeSize = Nullable()
            AskPrice = Nullable(d.Book.Ask.Price)
            AskSize = Nullable(d.Book.Ask.Size)
            BidPrice = Nullable(d.Book.Bid.Price)
            BidSize = Nullable(d.Book.Bid.Size)
        }
    else
        { 
            TimeStamp = d.TimeStamp.ToDateTime()
            TradePrice = Nullable(d.Trade.Price)
            TradeSize = Nullable(d.Trade.Size)
            AskPrice = Nullable()
            AskSize = Nullable()
            BidPrice = Nullable()
            BidSize = Nullable()
        }

let write data =
    let writeRecords (csv : CsvWriter) i =
        csv.WriteRecords(i)
    let write' (writer : StreamWriter )  i =
        using (new CsvWriter(writer, CultureInfo.InvariantCulture)) (fun csv -> writeRecords csv i )
    using (new StreamWriter($"L1L2_{ticker}.csv")) (fun writer -> write' writer data)


data
|> Seq.choose choose
|> Seq.map convert
|> write